In [1]:
import os
import re
import pandas as pd
import requests
import base64
import subprocess
from IPython.display import display, Image
from PIL import Image as PILImage

In [2]:
import keras_ocr

In [2]:
path = os.path.dirname(os.getcwd()) # Parent directory
image_folder = path+'/data/lines'

# Enocde Images

In [4]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [5]:
images = []
for image in os.listdir(image_folder):
    if image.endswith('.jpg'):
        images.append(image)

rows = []
for image in images:
    name = image.split('.')[0]
    name_split = name.split('_')[0]
    file_name = name_split.split('example')[1]
    line_name = name.split('_')[1]
    encoded_value = encode_image(image_folder+'/'+image)
    rows.append({'file': file_name, 'line': line_name, 'encoded': encoded_value})

images_encoded = pd.DataFrame(rows)

In [6]:
images_encoded['file'] = images_encoded['file'].astype('int')
images_encoded['line'] = images_encoded['line'].astype('int')
images_encoded = images_encoded.sort_values(by=['file', 'line']).reset_index(drop=True)
images_encoded['id'] = images_encoded['file'].astype(str) + '_' + images_encoded['line'].astype(str)
images_encoded.head(30)

,file,line,encoded,id
0,1,0,/9j/4QczRXhpZgAATU0AKgAAAAgADQEAAAMAAAABD5YAAA...,1_0
1,1,1,/9j/4QZBRXhpZgAATU0AKgAAAAgADQEAAAMAAAABD5YAAA...,1_1
2,1,2,/9j/4QX5RXhpZgAATU0AKgAAAAgADQEAAAMAAAABD5YAAA...,1_2
3,1,3,/9j/4QYvRXhpZgAATU0AKgAAAAgADQEAAAMAAAABD5YAAA...,1_3
4,1,4,/9j/4QZLRXhpZgAATU0AKgAAAAgADQEAAAMAAAABD5YAAA...,1_4
5,1,5,/9j/4QVaRXhpZgAATU0AKgAAAAgADQEAAAMAAAABD5YAAA...,1_5
6,1,6,/9j/4QVqRXhpZgAATU0AKgAAAAgADQEAAAMAAAABD5YAAA...,1_6
7,1,7,/9j/4QX5RXhpZgAATU0AKgAAAAgADQEAAAMAAAABD5YAAA...,1_7
8,1,8,/9j/4QWRRXhpZgAATU0AKgAAAAgADQEAAAMAAAABD5YAAA...,1_8
9,1,9,/9j/4QZyRXhpZgAATU0AKgAAAAgADQEAAAMAAAABD5YAAA...,1_9


# Keras
* Please check if the code is good

In [7]:
def kerasOCR(image_path):
    pipeline = keras_ocr.pipeline.Pipeline()
    image = keras_ocr.tools.read(image_path)
    prediction_groups = pipeline.recognize([image])
    words = []
    for line in prediction_groups[0]:
        for word in line:
            try:
                if isinstance(word[0], str):
                    words.append(word[0])
            except IndexError:
                continue
    text = ' '.join(words)
    return text

In [8]:
kerasOCR_output = {}
for image in os.listdir(image_folder):
    if image.endswith('.jpg'):
        image_path = image_folder + '/' + image
        text = kerasOCR(image_path)
        name = image.split('.')[0]
        name = name.split('example')[1]
        kerasOCR_output[name] = text

Looking for C:\Users\julien\.keras-ocr\craft_mlt_25k.h5

Instructions for updating:
Use `tf.image.resize(...method=ResizeMethod.BILINEAR...)` instead.

Looking for C:\Users\julien\.keras-ocr\crnn_kurapan.h5
3/3 [==============================] - 2s 410ms/step
Looking for C:\Users\julien\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\julien\.keras-ocr\crnn_kurapan.h5
1/1 [==============================] - 1s 1s/step
Looking for C:\Users\julien\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\julien\.keras-ocr\crnn_kurapan.h5
1/1 [==============================] - 1s 1s/step
Looking for C:\Users\julien\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\julien\.keras-ocr\crnn_kurapan.h5
1/1 [==============================] - 1s 1s/step
Looking for C:\Users\julien\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\julien\.keras-ocr\crnn_kurapan.h5
1/1 [==============================] - 2s 2s/step
Looking for C:\Users\julien\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\julien\.keras-ocr\c

In [9]:
kerasOCR_output_df = pd.DataFrame(kerasOCR_output.items(), columns=['file', 'text'])
kerasOCR_output_df[['file_name', 'line_name']] = kerasOCR_output_df['file'].str.split('_', expand=True)
kerasOCR_output_df[['file_name', 'line_name']] = kerasOCR_output_df[['file_name', 'line_name']].astype(int)
kerasOCR_output_df = kerasOCR_output_df.sort_values(by=['file_name', 'line_name']).reset_index(drop=True)
kerasOCR_output_df['text'] = kerasOCR_output_df['text'].replace(['\n', '\t'], ' ', regex=True)
kerasOCR_output_df['id'] = kerasOCR_output_df['file_name'].astype(str) + '_' + kerasOCR_output_df['line_name'].astype(str)
kerasOCR_output_df

,file,text,file_name,line_name,id
0,1_00,d d d s n r d u r d d d a d p o j d d p f d s ...,1,0,1_0
1,1_01,d r p o a g,1,1,1_1
2,1_02,d c b v f l,1,2,1_2
3,1_03,h 3 b b t d o g y w o 2 f j h r 1 s o n,1,3,1_3
4,1_04,g d j g b t t e f a g i,1,4,1_4
...,...,...,...,...,...
278,20_09,s 1 w d l n c o h o c m l o 1 1,20,9,20_9
279,20_10,o r s s g s s d j j j t i,20,10,20_10
280,20_11,g a o n d,20,11,20_11
281,20_12,b m i d,20,12,20_12


In [10]:
kerasOCR_output_df.to_csv(path+'/results/postprocessed/kerasOCR_perline_output.csv', index=False)

# Calculate Scores

In [3]:
df = pd.read_csv(path+'/data/transcription_perline_text.csv')

In [4]:
from evaluate import load

cer_metric =load("cer")
bleu_metric = load("bleu")  

In [5]:
kerasOcr = pd.read_csv(path+'/results/postprocessed/kerasOCR_perline_output.csv')

In [23]:
from evaluate import load

cer_metric =load("cer")
bleu_metric = load("bleu")  

bleu_gpt = {}
cer_gpt = {}

# Iterate over unique IDs
for id in df['id'].unique():
    # Extract the text as a single string, not as an array
    pred_text = kerasOcr[kerasOcr['id'] == id]['text'].values
    ref_text = df[df['id'] == id]['text'].values

    # Ensure the predictions and references are passed as a list of strings
    if len(pred_text) > 0 and len(ref_text) > 0:  # Check if both texts are not empty
        pred_text = pred_text[0]
        ref_text = ref_text[0]

        # Check for NaN values and strip whitespace
        if pd.notna(pred_text) and pd.notna(ref_text):
            pred_text = pred_text.strip()
            ref_text = ref_text.strip()

            # Ensure texts are not empty after stripping
            if pred_text and ref_text:
                bleu_gpt[id] = bleu_metric.compute(predictions=[pred_text], references=[ref_text])
                cer_gpt[id] = cer_metric.compute(predictions=[pred_text], references=[ref_text])
            else:
                bleu_gpt[id] = {'bleu': 0.0}  # Assign a default value if texts are empty
                cer_gpt[id] = 1.0  # Assign a default value if texts are empty
        else:
            bleu_gpt[id] = {'bleu': 0.0}  # Assign a default value if texts are NaN
            cer_gpt[id] = 1.0  # Assign a default value if texts are NaN
    else:
        bleu_gpt[id] = {'bleu': 0.0}  # Assign a default value if texts are empty
        cer_gpt[id] = 1.0  # Assign a default value if texts are empty

In [24]:
cer_gpt

{'1_0': 0.8228782287822878,
 '1_1': 0.8536585365853658,
 '1_2': 0.8536585365853658,
 '1_3': 0.8222222222222222,
 '1_4': 0.8505747126436781,
 '1_5': 0.8648648648648649,
 '1_6': 0.825,
 '1_7': 0.8571428571428571,
 '1_8': 0.8222222222222222,
 '1_9': 0.8,
 '1_10': 0.8,
 '1_11': 0.7870370370370371,
 '1_12': 0.7837837837837838,
 '1_13': 0.7615384615384615,
 '2_0': 0.8290282902829028,
 '2_1': 1.0,
 '2_2': 0.8041958041958042,
 '2_3': 0.8118811881188119,
 '2_4': 0.8037383177570093,
 '2_5': 0.8421052631578947,
 '2_6': 0.819672131147541,
 '2_7': 0.8333333333333334,
 '2_8': 0.816,
 '2_9': 0.8055555555555556,
 '2_10': 0.8571428571428571,
 '2_11': 0.8016528925619835,
 '2_12': 0.77,
 '2_13': 0.8333333333333334,
 '2_14': 0.752,
 '3_0': 0.8327183271832719,
 '3_1': 0.8018018018018018,
 '3_2': 0.7815126050420168,
 '3_3': 0.8191489361702128,
 '3_4': 0.8055555555555556,
 '3_5': 0.8222222222222222,
 '3_6': 0.8018867924528302,
 '3_7': 0.7803030303030303,
 '3_8': 0.8095238095238095,
 '3_9': 0.8285714285714286

In [25]:
bleu_gpt = pd.DataFrame(bleu_gpt).T
print(bleu_gpt['bleu'].mean(), bleu_gpt['bleu'].var())

0.0 0.0


In [26]:
bleu_gpt['id'] = bleu_gpt.index
bleu_gpt

,bleu,precisions,brevity_penalty,length_ratio,translation_length,reference_length,id
1_0,0.0,"[0.0, 0.0, 0.0, 0.0]",0.473726,0.572368,87,152,1_0
1_1,0.0,"[0.0, 0.0, 0.0, 0.0]",0.846482,0.857143,6,7,1_1
1_2,0.0,"[0.0, 0.0, 0.0, 0.0]",0.846482,0.857143,6,7,1_2
1_3,0.0,"[0.0, 0.0, 0.0, 0.0]",0.704688,0.740741,20,27,1_3
1_4,0.0,"[0.08333333333333333, 0.0, 0.0, 0.0]",0.606531,0.666667,12,18,1_4
...,...,...,...,...,...,...,...
20_9,0.0,"[0.0625, 0.0, 0.0, 0.0]",0.645649,0.695652,16,23,20_9
20_10,0.0,"[0.07692307692307693, 0.0, 0.0, 0.0]",0.50042,0.590909,13,22,20_10
20_11,0.0,"[0.0, 0.0, 0.0, 0.0]",0.67032,0.714286,5,7,20_11
20_12,0.0,"[0.0, 0.0, 0.0, 0.0]",0.606531,0.666667,4,6,20_12


In [27]:
bleu_gpt['file'] = bleu_gpt['id'].str.split('_').str[0].astype(int)
bleu_gpt.groupby('file')['bleu'].agg(['mean', 'var'])

,mean,var
file,,
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,0.0
8,0.0,0.0
9,0.0,0.0


In [29]:
bleu_gpt.to_csv(path+'/results/scores_comparisons/eval/bleu_kerasOCR_perline.csv')

In [28]:
cer_gpt = pd.DataFrame(cer_gpt.items(), columns=['id', 'cer'])

In [29]:
cer_gpt.head(50)

,id,cer
0,1_0,0.822878
1,1_1,0.853659
2,1_2,0.853659
3,1_3,0.822222
4,1_4,0.850575
5,1_5,0.864865
6,1_6,0.825000
7,1_7,0.857143
8,1_8,0.822222
9,1_9,0.800000


In [30]:
cer_gpt.to_csv(path+'/results/scores_comparisons/eval/cer_kerasOCR_perline.csv')

In [31]:
cer_gpt['file'] = cer_gpt['id'].str.split('_').str[0].astype(int)
cer_gpt.groupby('file')['cer'].agg(['mean', 'var'])

,mean,var
file,,
1,0.821756,0.001016
2,0.825309,0.003039
3,0.863897,0.033923
4,0.842386,0.006141
5,0.834935,0.003336
6,0.839017,0.002563
7,0.820444,0.000803
8,0.824706,0.000774
9,0.849523,0.001462
